In [63]:
import tensorflow as tf
import student_generator as sg
import numpy as np
import pandas as pd

test_size = 20

students = [sg.generate_students(60) for i in range(test_size)]

students_copy = [students[i].copy() for i in range(test_size)]

priv_students = [sg.choose_priv_students(
    students[i]) for i in range(test_size)]

non_priv_students = [sg.choose_non_priv_students(
    students_copy[i]) for i in range(test_size)]

x_test_priv = [priv_students[i][:, 0:4] for i in range(test_size)]
y_test_priv = [priv_students[i][:, 5] for i in range(test_size)]

x_test_non_priv = [non_priv_students[i][:, 0:4] for i in range(test_size)]
y_test_non_priv = [non_priv_students[i][:, 5] for i in range(test_size)]


In [64]:
x_test_data = np.reshape(x_test_priv, (test_size * 1500, 4))
y_test_priv_data = np.reshape(y_test_priv, (test_size * 1500, 1))

x_test_non_priv_data = np.reshape(x_test_non_priv, (test_size * 1500, 4))
y_test_non_priv_data = np.reshape(y_test_non_priv, (test_size * 1500, 1))

In [65]:
model_priv = tf.keras.models.load_model('models/double_models/model_priv.h5')
model_non_priv = tf.keras.models.load_model('models/double_models/model_non_priv.h5')

In [66]:
model_priv.evaluate(x_test_data, y_test_priv_data)
model_non_priv.evaluate(x_test_non_priv_data, y_test_non_priv_data)

938/938 [==============================] - 2s 2ms/step - loss: 0.0818 - accuracy: 0.9589


[0.08180725574493408, 0.958899974822998]

In [69]:
predictions_priv = model_priv.predict(x_test_data[0:1500])
predictions_non_priv = model_non_priv.predict(x_test_non_priv_data[0:1500])

expected_priv = y_test_priv_data[0:1500]
expected_non_priv = y_test_non_priv_data[0:1500]


predictions_priv = predictions_priv.round().reshape(1500)
predictions_non_priv = predictions_non_priv.round().reshape(1500)

expected_priv = expected_priv.reshape(1500)
expected_non_priv = expected_non_priv.reshape(1500)

# to excel priv and expected
import pandas as pd

df = pd.DataFrame({'predicted': predictions_priv, 'excepted': expected_priv ,'math': x_test_data[0:1500, 0], 'en': x_test_data[0:1500, 1], 'ua': x_test_data[0:1500, 2], 'priv': x_test_data[0:1500, 3]})

df.to_excel('predicted_priv.xlsx', index=False)

# merge priv and non priv
predicted = [ predictions_priv[i] if students[0][i][3] == 1 else predictions_non_priv[i] for i in range(1500)]
expected = [ expected_priv[i] if students[0][i][3] == 1 else expected_non_priv[i] for i in range(1500)]

priveleged_right = 0
priveleged_wrong = 0

non_priveleged_right = 0
non_priveleged_wrong = 0

priveleged_right_accepted = 0
non_priveleged_right_accepted = 0

for (pred, exp, test) in zip(predicted, expected, x_test_data[0:1500]):
    if test[3] == 1:
        if pred == exp:
            priveleged_right += 1
            if pred == 1:
                priveleged_right_accepted += 1
        else:
            priveleged_wrong += 1
    else:
        if pred == exp:
            non_priveleged_right += 1
            if pred == 1:
                non_priveleged_right_accepted += 1
        else:
            non_priveleged_wrong += 1

print("Priveleged right: ", priveleged_right)
print("Priveleged wrong: ", priveleged_wrong)
print("Non priveleged right: ", non_priveleged_right)
print("Non priveleged wrong: ", non_priveleged_wrong)

print("Priveleged right accepted: ", priveleged_right_accepted)
print("Non priveleged right accepted: ", non_priveleged_right_accepted)

# to excel

import pandas as pd

df = pd.DataFrame({'predicted': predicted, 'excepted': expected, 'math': x_test_data[0:1500, 0], 'en': x_test_data[0:1500, 1], 'ua': x_test_data[0:1500, 2], 'priv': x_test_data[0:1500, 3]})

df.to_excel('predicted.xlsx', index=False)

47/47 [==============================] - 0s 2ms/step
Priveleged right:  50
Priveleged wrong:  10
Non priveleged right:  1372
Non priveleged wrong:  68
Priveleged right accepted:  35
Non priveleged right accepted:  350
